In [26]:
# Установим библиотеку nest_asyncio
#!pip install nest_asyncio
import nest_asyncio

nest_asyncio.apply()
from openai import OpenAI
import os
import getpass
import pandas as pd
import openai  # будем использовать для токинизации
from typing import Tuple, List
from scipy import spatial  # вычисляет сходство векторов
import tiktoken


In [37]:
! unzip automobile_file.zip

Archive:  automobile_file.zip
  inflating: automobile.csv          
  inflating: automobile.pickle       


In [2]:
EMBEDDING_MODEL = "text-embedding-ada-002"  # Модель токенизации от OpenAI
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use


In [34]:
df = pd.DataFrame()
import pickle
with open('automobile.pickle', 'rb') as f: 
    df = pickle.load(f)
print(f"Loaded {df.shape} rows")

Loaded (5384, 2) rows


In [25]:
def relatedness_fn2(x, y):
    # print(f"x={type(x)}")
    # print(f"y={type(y)}")
    
    return 1 - spatial.distance.cosine(x, y)
    
# Функция поиска
def strings_ranked_by_relatedness(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    relatedness_fn=relatedness_fn2, # функция схожести, косинусное расстояние
    top_n: int = 100 # выбор лучших n-результатов
) -> Tuple[List[str], List[float]]: # Функция возвращает кортеж двух списков, первый содержит строки, второй - числа с плавающей запятой
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""
    print(f"query='{query}'")
    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )

    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding
    #print(f"query_embedding='{query_embedding}'")
    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]

In [23]:
#strings_ranked_by_relatedness("fast car speed", df, top_n=1)

query='fast car speed'


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>
x=<class 'list'>
y=<class 'list'>


(('NZ Performance Car\n\n==Magazine contents==\n\n===Modified car features===\n\nThe extent that a car is required to be modified has steadily increased since the first issues. While 300&nbsp;hp was considered to be a huge amount of power when the magazine started, 300&nbsp;kW may not even secure a spot as a feature car, unless the rest of the car is outstanding or unusual.\n\nThis is not to say that NZ Performance Car solely focuses on power, but power figures are part of a more holistic approach considered when modifying a car.',),
 (0.8000849509620129,))

In [7]:
os.environ["API_TOKEN"] = getpass.getpass("Введите Telegram API Key:")

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

In [9]:
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

In [27]:
# с этой функцией мы уже знакомы
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке для заданной модели"""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция формирования запроса к chatGPT по пользовательскому вопросу и базе знаний
def query_message(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    model: str, # модель
    token_budget: int # ограничение на число отсылаемых токенов в модель
) -> str:
    """Возвращает сообщение для GPT с соответствующими исходными текстами, извлеченными из фрейма данных (базы знаний)."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df) # функция ранжирования базы знаний по пользовательскому запросу
    # Шаблон инструкции для chatGPT
    message = 'Use the below articles on Automotive industry to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    # Шаблон для вопроса
    question = f"\n\nQuestion: {query}"

    # Добавляем к сообщению для chatGPT релевантные строки из базы знаний, пока не выйдем за допустимое число токенов
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (num_tokens(message + next_article + question, model=model) > token_budget):
            break
        else:
            message += next_article
    return message + question

def ask(
    query: str, # пользовательский запрос
    df: pd.DataFrame = df, # DataFrame со столбцами text и embedding (база знаний)
    model: str = GPT_MODEL, # модель
    token_budget: int = 4096 - 500, # ограничение на число отсылаемых токенов в модель
    print_message: bool = False, # нужно ли выводить сообщение перед отправкой
) -> str:
    """Отвечает на вопрос, используя GPT и базу знаний."""
    # Формируем сообщение к chatGPT (функция выше)
    message = query_message(query, df, model=model, token_budget=token_budget)
    # Если параметр True, то выводим сообщение
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about Automotive industry."},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0 # гиперпараметр степени случайности при генерации текста. Влияет на то, как модель выбирает следующее слово в последовательности.
    )
    response_message = response.choices[0].message.content
    return response_message

In [29]:
ask("what is it Automotive industry ?")

query='what is it Automotive industry ?'


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'The Automotive industry refers to the sector involved in the design, development, manufacturing, marketing, and selling of motor vehicles. This industry encompasses a wide range of activities related to automobiles, including car manufacturers, parts suppliers, dealerships, repair and maintenance services, and more. It plays a significant role in the global economy and is constantly evolving with advancements in technology, sustainability, and consumer preferences.'

In [30]:
async def find_relevant_text(question_str) -> str:
    strings, relatednesses = strings_ranked_by_relatedness(question_str, df, top_n=1)
    
    for string, relatedness in zip(strings, relatednesses):
        print(f"{relatedness=:.3f}")
        print(string)
        return string
    return False

#print(await find_relevant_text("what is the best vehicle in the world?"))c

In [35]:
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command

# Включаем логирование, чтобы не пропустить важные сообщения
logging.basicConfig(level=logging.INFO)

# Замените "YOUR_BOT_TOKEN" на токен, который вы получили от BotFather
API_TOKEN = os.environ["API_TOKEN"] 


# Объект бота
bot = Bot(token=API_TOKEN)
# Диспетчер
dp = Dispatcher()

# Хэндлер на команду /start
@dp.message(Command("help"))
async def cmd_help(message: types.Message):
    await message.answer("The theme of this knowledge base is 'Automotive industry'")
    await message.answer(f"Current size of knowledge base is {df.shape[0]} articles")
    await message.answer(f"To ask question - you will need to send message with 'Question:' prefix.")
    await message.answer(f"For example 'Question: what is the best vehicle in the world?' ")

# Хэндлер на команду /start
@dp.message()
async def cmd_main(message: types.Message):

    if message.text.lower().startswith("question:"):
        print(message.text)
        question_edited = message.text.lower().replace("question:", "").strip()
        result = ask(question_edited)
        await message.answer(result)
        return
    await message.answer("Use /help command to see the available commands")
    
# Запуск процесса поллинга новых апдейтов
async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())


INFO:aiogram.dispatcher:Start polling
INFO:aiogram.dispatcher:Run polling for bot @bss_test_tg_kv_bot id=6387388299 - 'bss_test_tg'


Question:  What is the difference between Honda Civic and Toyota Supra?
query='what is the difference between honda civic and toyota supra?'


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:aiogram.event:Update id=266353521 is handled. Duration 5330 ms by bot id=6387388299


Question:  What is the difference between Honda Civic and Toyota Supra?
query='what is the difference between honda civic and toyota supra?'


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:aiogram.event:Update id=266353522 is handled. Duration 4966 ms by bot id=6387388299
INFO:aiogram.event:Update id=266353523 is handled. Duration 3129 ms by bot id=6387388299
INFO:aiogram.dispatcher:Polling stopped for bot @bss_test_tg_kv_bot id=6387388299 - 'bss_test_tg'
INFO:aiogram.dispatcher:Polling stopped
